In [1]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def firsts(list):
    answer=[]
    for pair in list:
        answer=answer+[pair[0]]
    return answer

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jLeoPowerConstantTerm(m,power,multiplier):
    trun=jLeo(power,m,multiplier)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def last_element(listo):return listo[len(listo)-1]
    
#Broadhurst code from #A005148:

#? {nt=100; a=[1]; b=[1]; d=1; e=0; g=0; print(1); for(n=2, nt, c=48*(a[n-1]+g)+128*(d-32*e); 
#e=d; d=c; i=(n-1)\2; g=12*if(n%2==0, a[n/2]^2)+24*sum(j=1, i, 
#a[j]*a[n-j]); h=12*if(n%2==0, b[n/2]^2)+24*sum(j=1, i, b[j]*b[n-j]); f=(c+5*h)/n^2-g; a=concat(a, f); 
#b=concat(b, n*f); print(f))}

#produces this on the PARI online evaluator (I have inserted the commas):
br=[
1,
47,
2488,
138799,
7976456,
467232200,
27736348480,
1662803271215,
100442427373480,
6103747246289272,
372725876150863808,
22852464771010647496,
1405886026610765892544,
86741060172969340021952,
5365190340823180439326208,
332577246704242939511725615,
20655377769544663820919905000,
1285027807539621869480480977880,
80066610886753513409821525593280,
4995543732566526565060187887772024,
312067903389730540416319245145039936,
19516459352109724206910675815791735872,
1221787478073080268912138739833447254528,
76558881238278398609546573647116818306504,
4801399849802188285872546222298724299377856,
301358552889212442951924121355286655092791360,
18928524108186605379268259069278244869735006720,
1189719542605042010945455887482239233732751142080,
74824958481405101799295401923145498080031496317440,
4708731584940969251488540213411242070133095720768000,
296483323638911778793802123013217365155428610625064960,
18677571039055424502042574350078071038555962934810664495,
1177200955467256907707767829606512556434525730284672082280,
74229820742983998523807878655148660941364964757170232076440,
4682657672641000613276353688819373189604961982881761635174080,
295516785862704112676947743865736338547152307208873658542187480,
18656838683258040776726836797753969443154060448210951169536087360,
1178287550937265649491805466460363896744099593833261406542090821440,
74441259433548426510664621182339422182178689134172479673100078686720,
4704546876230537649051669928635037299315044055233418643313504347890040,
297410696380227510473584821926459754598587577997951261584830786025989440,
18807176292551896455842616399574167855948518855982280636468413444438841280,
1189632505858785415664268185396568316810012962868095237190924015678644805120,
75269436592700558660145646818728077669744495747378078929068356710829357904960,
4763606735739477078702262301306618196904330454342036172567804617626114845601280,
301550219357655322958904198748139655940272138707157414253528789096123355242370560,
19093491105382437947961430595496009051927469794600124607374594862297809973497425920,
1209229421833128214532165231904398024088456532579184673374765702204525386892709582280,
76599462222171488217469562807555444840329820375936645628428503967599842536403748392640,
4853249476279584943018752544135518205835823652569328104071808597099976302206777672382272,
307557161322636240130429378891312616501367843861094990590342588813640658716687276702796288,
19494049310445477564550759598016449348244789590449694853790210431694353584090244058674665536,
1235828235382992966112000337356874542591602729215314383988292324949451319922660010288507026944,
78359386541981263753587663605771220395173695760309511724942887619218844604408365062990458445312,
4969331730256147172835336645727281480799885420753251882375600240456687485460922105379475213365248,
315192836223790555211163444188645825759965673786927108224693473418739558121668975477385609064230080,
19995098780164839843271975968833674908892528077762766665489021901583285029365260596621852077438476800,
1268636632413268672729488723673519658349225434504337043331170274695193536624267473803997222986457986560,
80503572319005599896519802085748511261953758070993177770252662999751870126880338970620619030455487016960,
5109226023515231795174271806642022238278119546486922652746346463140324125835625762209193286894097317188096,
324306079896787161344357124063601140357997183120543361083780244695916059297903737976316852087325736124084224,
20587951238469144728875413572545165973124146169862711202444824611729329948630917798996527215521428049827442688,
1307155886598940172607280751052393343723376964347599361863473016335357927316478471270092962849033836059872428032,
83003437890819298061159220674050513337514687509034267017635549288525596646068731914307086031798152640655708675631,
5271297646867701243231620056348828996858362139183014747261830107569871776680492045283621280463257824831450361228904,
334803625831811164273307471398151444570380979015726639675077730502474540099263374815508109958714263034979870045590040,
21267301347069209180434546119316800827596753601228462222166318226579132663526299067285136882052633679235269979706997440,
1351085301766957831759285167329838127692235945761375100182242807152550787271816136969058195839695684514251809964546971800,
85842015014782910888737358332857326909867676530325317899274801087388530640486646412863052257542933472850576480889534912320,
5454594181103314395353884318297045062026694474478123129221277721208935931363965378785241897523306877686001373372245607417152,
346632363267943630179319892957146981236491202095299908808662717047804243504180203803669349935923567992160546743779715198734848,
22030211252839896655760044436342222596667157343136792244243996326108725373610974640845764035532176284454550965374859141605876696,
1400264005045227498693320089227035884825680282492704474103652886750979292873550448366299419926933438094105780187281431855082851904,
89010607580108363146057947796447019984406819117601908324673767705214888723143847657440514237648023072092581303730690468541358446272,
5658653431087770147783431973072442739707731119479989465299403213540757386414610090412270459203277370032300491403324236240673058231808,
359768282283136451938233183643880156808956764689158714079901896700007719423392215459057530585195812985027222576505521483746145927094080,
22875473496936829687402292014161718834412927763836095316924893494178776078421448526507427925257601899589958056800447532066174066611688960,
1454634188002381701560543002374509285000151863499255133979293536524595484779236558931041033869616374153934389337102738325693541500885040640,
92506668310411752697700027500541107599054510647853447997767226456825933551220718898889759216482942932408592031625495444638507054727309701120,
5883380647402071727818124358464357357061997088366913641247755161122708769301392216445412497947130792332367814389247083022141893681918673035640,
374209367072642703061850255891790471856520862005092473938011251040764312378697398309875157737634899606364054132972840806767260748645213681887040,
23803199270417251468737963122626707467816366171461192749563400557478169477201165362512851772159448598935076348960799452562007578125691327998310080,
1514217230105491006416286735192399705713778112569360921940868647785308212475854144353099514116567802127049424293324092083010142613089652094239352320,
96332413045374746200370571514086365134691986056054938357641223739177015618938978964850794817403946266783867698429445521201166592631737075434674509760,
6128968041798698464871983060993177701445048464359248049771600533082817745158394358756109989641756654653420473533147391458535031474858708357818557350400,
389970917973786025826826148232023527188983145106526374860563378061488842725970203994300386046448714269543873579140545202535929354547230426525139471434240,
24814546341676335548842726357075976008312393230217108797033854347814848781823641548406242672165084256308341602099263161325277248861481206883551599766753280,
1579097865621621078127966349210038916312383695762720347829578839794195077696910123713050602263102661325782769915208166009373762048848807388370046579798291520,
100493898862353192639599085772479279507605308104393463045761183781449039477596456154787934584841158653853713852389459365917827655606822211804400142188630504960,
6395841086583513721518708585123859851483827511052356442692606484016955654747071476807355907729334535301712240745743084597082858081715137457733426771133436244480,
407082422122816769108928976084224767774083074925265772652509091404070682745260601092117551854989868576150378265801324011663291789547097044490460289150388722831360,
25911536639811395996751656925990326528992266850810482361355730044302538357241007798389242049209348961586470395969302758332164645822911593670231053397678686836037120,
1649413547559894591700394303117536943729317363777978884992676745553087694293098678096611231647446457954863017134436134803435906819926459313995976199288693719229255680,
105000403824773770333649105789820480034335517728840968988334310643726824123735041790377778857595055184234699203779438530862092851011368656739154398110404364696677847040,
6684622341158487811999668275612452018461145981041862416511208589860402818472403829783593014653257049064920715626593324371966426230798803277918448269747031588460412436480,
425585443813049916737778209891287760908166309712249413374261870396364033955750883326966977701620958916811619741413201896787531939944010422188516605008297556440177053390280,
27096933232548903092613115617140470313230857234611666991627504070866204307527763605986822544358886296299818124185350044201435781986714935884426869011696899042312957811757760,
1725347275200146778788065707502165945462707429554800950293313680035690477290460889039398857128345130378453865634762702538349329424603778897699271642610584658725955102258591040,
109864008941653383497254010859226498595190791378492674731068039950918263737791469856053624392850573136737164626778125500188292591996295723267255817207828065983109119673561029120,
6996107097499132971946127571962655485323611430800974100070703369809020527942862074490096967477739069312314066266444140870911945979977377211097174725520015644037734486845682825024]

def broadhurst(n):return br[n-1]

def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

print("okay1")

okay1


In [ ]:
#output snipped; I interrupted this after k = 94.
polydata=[]
numericalfactordata=[]
broadhurst_values=[]
tests=[]
import time
for power in [1..100]:
    start=time.time()
    data=[]
    for m in [3..3*power+3]:
        cst= jpowerConstantTerm(m,power)
        data=data+[[m,cst]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[power,rl]]
    nf=ZZ(numericalfactor(rl))
    difference=nf/24-nszfunction(power)
    tests=tests+[difference]
    finish=time.time()
    print("(power,difference,time): ",(power,difference,finish-start))
print("===============================================================================")
print("tests:")
print(tests)
#below was originally done in a separate cell
#import pickle
#wfile = open('/Users/barrybrent/26oct22no1.txt','wb')
#t = pickle.dumps(str(polydata))
#wfile.write(t)
#wfile.close()
---------------------------------------------------------------------------------------------------
polydata=[]
numericalfactordata=[]
broadhurst_values=[]
tests=[]
import time
for power in [95..100]:
    start=time.time()
    data=[]
    for m in [3..3*power+3]:
        cst=QQ(jpowerConstantTerm(m,power))
        data=data+[[m,cst]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[power,rl]]
    nf=ZZ(numericalfactor(rl))
    difference=nf/24-broadhurst(power)
    tests=tests+[difference]
    finish=time.time()
    print("(power,difference,time): ",(power,difference,finish-start))
print("===============================================================================")
print("tests:")
print(tests)
import pickle
wfile = open('/Users/barrybrent/26oct22no2.txt','wb')
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()
print("tests:")
print(tests)

In [2]:
a=[1,2,3];b=[4,5,6];c=a+b;print(c)

[1, 2, 3, 4, 5, 6]


In [6]:
import pickle
rsfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rsfile)
rsfile.close()
s=stripQuotationMarks(rs)

rtfile = open('/Users/barrybrent/26oct22no2.txt','rb')
rt=pickle.load(rtfile)
rtfile.close()
t=stripQuotationMarks(rt)

u=s+t

wufile = open('/Users/barrybrent/5dec22no1.txt','wb')
u = pickle.dumps(str(u))
wufile.write(u)

1968604

In [8]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
print(len(s))

100


In [8]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
no=[]
no2=[]
no3=[]
no4=[]
for n in [1..100]:
    p=prime(n)
    firsts=[]
    for k in [0..len(s)-1]: 
        residues=[]
        data=[]
        power=s[k][0]
        poly=D(s[k][1]) # poly(m) = cst term of j_m^power
        nf=QQ(numericalfactor(poly))
        polyd=expand(poly/nf)
        for j in [3..103]: # nb
            m=p^j
            polydm=evaluate(polyd,m)
            o=order(p,polydm)
            data=data+[o]
            core=polydm/p^o
            mcp=mod(core,p)
            residues=residues+[mcp]
        first=data[0]
        firsts=firsts+[first]
        if data==[]:print("power: ",power," prime: ",p," empty")
        if data!=[]:
            dt=difference_table(data)
            ul=uniteList(dt)
            ulm=uniteList(residues)
            if ul[0]!=power:no=no+[(p,power)]
            if len(ul)>1:no2=no2+[(p,power)]
            if len(ul)==0:no3=no3+[(p,power)]
            if len(ulm)!=1:no4=no4+[(p,power)]
        print("n:",n," prime",p," power:",power," ul[0]-power:",ul[0]-power," ulm:",ulm," first:",first)
        
        
        
    print("firsts for p = ",p,":");print(firsts);print()
print("============================================================================================")
print("no:");print(no);print();print("no2:");print(no2);print();print("no3:");print(no3)
print("no4:");print(no4)

n: 1  prime 2  power: 1  ul[0]-power: 0  ulm: [1]  first: 5
n: 1  prime 2  power: 2  ul[0]-power: 0  ulm: [1]  first: 10
n: 1  prime 2  power: 3  ul[0]-power: 0  ulm: [1]  first: 15
n: 1  prime 2  power: 4  ul[0]-power: 0  ulm: [1]  first: 20
n: 1  prime 2  power: 5  ul[0]-power: 0  ulm: [1]  first: 25
n: 1  prime 2  power: 6  ul[0]-power: 0  ulm: [1]  first: 30
n: 1  prime 2  power: 7  ul[0]-power: 0  ulm: [1]  first: 35
n: 1  prime 2  power: 8  ul[0]-power: 0  ulm: [1]  first: 40
n: 1  prime 2  power: 9  ul[0]-power: 0  ulm: [1]  first: 45
n: 1  prime 2  power: 10  ul[0]-power: 0  ulm: [1]  first: 50
n: 1  prime 2  power: 11  ul[0]-power: 0  ulm: [1]  first: 55
n: 1  prime 2  power: 12  ul[0]-power: 0  ulm: [1]  first: 60
n: 1  prime 2  power: 13  ul[0]-power: 0  ulm: [1]  first: 65
n: 1  prime 2  power: 14  ul[0]-power: 0  ulm: [1]  first: 70
n: 1  prime 2  power: 15  ul[0]-power: 0  ulm: [1]  first: 75
n: 1  prime 2  power: 16  ul[0]-power: 0  ulm: [1]  first: 80
n: 1  prime 2  pow

n: 2  prime 3  power: 25  ul[0]-power: 0  ulm: [1]  first: 54
n: 2  prime 3  power: 26  ul[0]-power: 0  ulm: [1]  first: 57
n: 2  prime 3  power: 27  ul[0]-power: 0  ulm: [1]  first: 54
n: 2  prime 3  power: 28  ul[0]-power: 0  ulm: [2]  first: 57
n: 2  prime 3  power: 29  ul[0]-power: 0  ulm: [2]  first: 60
n: 2  prime 3  power: 30  ul[0]-power: 0  ulm: [2]  first: 61
n: 2  prime 3  power: 31  ul[0]-power: 0  ulm: [1]  first: 64
n: 2  prime 3  power: 32  ul[0]-power: 0  ulm: [1]  first: 67
n: 2  prime 3  power: 33  ul[0]-power: 0  ulm: [2]  first: 68
n: 2  prime 3  power: 34  ul[0]-power: 0  ulm: [2]  first: 71
n: 2  prime 3  power: 35  ul[0]-power: 0  ulm: [2]  first: 74
n: 2  prime 3  power: 36  ul[0]-power: 0  ulm: [2]  first: 73
n: 2  prime 3  power: 37  ul[0]-power: 0  ulm: [1]  first: 76
n: 2  prime 3  power: 38  ul[0]-power: 0  ulm: [1]  first: 79
n: 2  prime 3  power: 39  ul[0]-power: 0  ulm: [1]  first: 80
n: 2  prime 3  power: 40  ul[0]-power: 0  ulm: [2]  first: 83
n: 2  pr

n: 3  prime 5  power: 49  ul[0]-power: 0  ulm: [1]  first: 126
n: 3  prime 5  power: 50  ul[0]-power: 0  ulm: [1]  first: 127
n: 3  prime 5  power: 51  ul[0]-power: 0  ulm: [3]  first: 130
n: 3  prime 5  power: 52  ul[0]-power: 0  ulm: [1]  first: 134
n: 3  prime 5  power: 53  ul[0]-power: 0  ulm: [1]  first: 136
n: 3  prime 5  power: 54  ul[0]-power: 0  ulm: [4]  first: 139
n: 3  prime 5  power: 55  ul[0]-power: 0  ulm: [3]  first: 140
n: 3  prime 5  power: 56  ul[0]-power: 0  ulm: [3]  first: 142
n: 3  prime 5  power: 57  ul[0]-power: 0  ulm: [4]  first: 145
n: 3  prime 5  power: 58  ul[0]-power: 0  ulm: [4]  first: 148
n: 3  prime 5  power: 59  ul[0]-power: 0  ulm: [3]  first: 151
n: 3  prime 5  power: 60  ul[0]-power: 0  ulm: [1]  first: 154
n: 3  prime 5  power: 61  ul[0]-power: 0  ulm: [3]  first: 157
n: 3  prime 5  power: 62  ul[0]-power: 0  ulm: [1]  first: 161
n: 3  prime 5  power: 63  ul[0]-power: 0  ulm: [1]  first: 163
n: 3  prime 5  power: 64  ul[0]-power: 0  ulm: [4]  fir

n: 4  prime 7  power: 73  ul[0]-power: 0  ulm: [5]  first: 197
n: 4  prime 7  power: 74  ul[0]-power: 0  ulm: [1]  first: 200
n: 4  prime 7  power: 75  ul[0]-power: 0  ulm: [2]  first: 203
n: 4  prime 7  power: 76  ul[0]-power: 0  ulm: [6]  first: 207
n: 4  prime 7  power: 77  ul[0]-power: 0  ulm: [3]  first: 207
n: 4  prime 7  power: 78  ul[0]-power: 0  ulm: [6]  first: 210
n: 4  prime 7  power: 79  ul[0]-power: 0  ulm: [3]  first: 213
n: 4  prime 7  power: 80  ul[0]-power: 0  ulm: [4]  first: 216
n: 4  prime 7  power: 81  ul[0]-power: 0  ulm: [5]  first: 219
n: 4  prime 7  power: 82  ul[0]-power: 0  ulm: [3]  first: 222
n: 4  prime 7  power: 83  ul[0]-power: 0  ulm: [5]  first: 226
n: 4  prime 7  power: 84  ul[0]-power: 0  ulm: [6]  first: 226
n: 4  prime 7  power: 85  ul[0]-power: 0  ulm: [4]  first: 229
n: 4  prime 7  power: 86  ul[0]-power: 0  ulm: [2]  first: 232
n: 4  prime 7  power: 87  ul[0]-power: 0  ulm: [5]  first: 235
n: 4  prime 7  power: 88  ul[0]-power: 0  ulm: [1]  fir

n: 5  prime 11  power: 95  ul[0]-power: 0  ulm: [10]  first: 269
n: 5  prime 11  power: 96  ul[0]-power: 0  ulm: [3]  first: 272
n: 5  prime 11  power: 97  ul[0]-power: 0  ulm: [3]  first: 275
n: 5  prime 11  power: 98  ul[0]-power: 0  ulm: [3]  first: 278
n: 5  prime 11  power: 99  ul[0]-power: 0  ulm: [9]  first: 279
n: 5  prime 11  power: 100  ul[0]-power: 0  ulm: [1]  first: 281
firsts for p =  11 :
[3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 31, 34, 36, 40, 43, 46, 49, 51, 55, 58, 61, 62, 65, 68, 71, 74, 77, 80, 83, 86, 89, 92, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 124, 127, 130, 132, 136, 139, 142, 145, 147, 151, 154, 155, 158, 161, 162, 167, 170, 173, 176, 178, 182, 185, 186, 189, 192, 195, 198, 201, 204, 207, 210, 213, 216, 217, 220, 223, 226, 229, 232, 235, 238, 241, 244, 247, 248, 251, 254, 257, 260, 263, 266, 269, 272, 275, 278, 279, 281]

n: 6  prime 13  power: 1  ul[0]-power: 0  ulm: [10]  first: 3
n: 6  prime 13  power: 2  ul[0]-power: 0  ulm: [12]  first: 6
n: 6 

...

n: 95  prime 499  power: 96  ul[0]-power: 0  ulm: [412]  first: 288
n: 95  prime 499  power: 97  ul[0]-power: 0  ulm: [156]  first: 291
n: 95  prime 499  power: 98  ul[0]-power: 0  ulm: [402]  first: 294
n: 95  prime 499  power: 99  ul[0]-power: 0  ulm: [111]  first: 297
n: 95  prime 499  power: 100  ul[0]-power: 0  ulm: [114]  first: 300
firsts for p =  499 :
[3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 71, 75, 78, 81, 84, 87, 90, 93, 96, 99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135, 138, 141, 144, 147, 150, 153, 156, 159, 162, 165, 168, 171, 174, 177, 180, 183, 186, 189, 192, 195, 198, 201, 204, 207, 210, 213, 216, 219, 222, 225, 228, 231, 234, 237, 240, 243, 246, 249, 252, 255, 258, 261, 264, 267, 270, 273, 276, 279, 282, 285, 288, 291, 294, 297, 300]

n: 96  prime 503  power: 1  ul[0]-power: 0  ulm: [169]  first: 3
n: 96  prime 503  power: 2  ul[0]-power: 0  ulm: [500]  first: 6
n: 96  prime 503  power: 3  ul[0]-power: 

n: 97  prime 509  power: 6  ul[0]-power: 0  ulm: [44]  first: 18
n: 97  prime 509  power: 7  ul[0]-power: 0  ulm: [125]  first: 21
n: 97  prime 509  power: 8  ul[0]-power: 0  ulm: [380]  first: 24
n: 97  prime 509  power: 9  ul[0]-power: 0  ulm: [318]  first: 27
n: 97  prime 509  power: 10  ul[0]-power: 0  ulm: [70]  first: 30
n: 97  prime 509  power: 11  ul[0]-power: 0  ulm: [300]  first: 33
n: 97  prime 509  power: 12  ul[0]-power: 0  ulm: [196]  first: 36
n: 97  prime 509  power: 13  ul[0]-power: 0  ulm: [315]  first: 39
n: 97  prime 509  power: 14  ul[0]-power: 0  ulm: [381]  first: 42
n: 97  prime 509  power: 15  ul[0]-power: 0  ulm: [324]  first: 45
n: 97  prime 509  power: 16  ul[0]-power: 0  ulm: [295]  first: 48
n: 97  prime 509  power: 17  ul[0]-power: 0  ulm: [426]  first: 51
n: 97  prime 509  power: 18  ul[0]-power: 0  ulm: [84]  first: 54
n: 97  prime 509  power: 19  ul[0]-power: 0  ulm: [75]  first: 57
n: 97  prime 509  power: 20  ul[0]-power: 0  ulm: [252]  first: 60
n: 

n: 98  prime 521  power: 21  ul[0]-power: 0  ulm: [437]  first: 63
n: 98  prime 521  power: 22  ul[0]-power: 0  ulm: [38]  first: 66
n: 98  prime 521  power: 23  ul[0]-power: 0  ulm: [315]  first: 69
n: 98  prime 521  power: 24  ul[0]-power: 0  ulm: [370]  first: 72
n: 98  prime 521  power: 25  ul[0]-power: 0  ulm: [141]  first: 75
n: 98  prime 521  power: 26  ul[0]-power: 0  ulm: [472]  first: 78
n: 98  prime 521  power: 27  ul[0]-power: 0  ulm: [320]  first: 81
n: 98  prime 521  power: 28  ul[0]-power: 0  ulm: [238]  first: 84
n: 98  prime 521  power: 29  ul[0]-power: 0  ulm: [49]  first: 87
n: 98  prime 521  power: 30  ul[0]-power: 0  ulm: [292]  first: 90
n: 98  prime 521  power: 31  ul[0]-power: 0  ulm: [300]  first: 93
n: 98  prime 521  power: 32  ul[0]-power: 0  ulm: [218]  first: 96
n: 98  prime 521  power: 33  ul[0]-power: 0  ulm: [70]  first: 99
n: 98  prime 521  power: 34  ul[0]-power: 0  ulm: [499]  first: 102
n: 98  prime 521  power: 35  ul[0]-power: 0  ulm: [466]  first: 

n: 99  prime 523  power: 36  ul[0]-power: 0  ulm: [183]  first: 108
n: 99  prime 523  power: 37  ul[0]-power: 0  ulm: [391]  first: 111
n: 99  prime 523  power: 38  ul[0]-power: 0  ulm: [244]  first: 114
n: 99  prime 523  power: 39  ul[0]-power: 0  ulm: [81]  first: 117
n: 99  prime 523  power: 40  ul[0]-power: 0  ulm: [76]  first: 120
n: 99  prime 523  power: 41  ul[0]-power: 0  ulm: [55]  first: 123
n: 99  prime 523  power: 42  ul[0]-power: 0  ulm: [84]  first: 126
n: 99  prime 523  power: 43  ul[0]-power: 0  ulm: [479]  first: 129
n: 99  prime 523  power: 44  ul[0]-power: 0  ulm: [87]  first: 132
n: 99  prime 523  power: 45  ul[0]-power: 0  ulm: [438]  first: 135
n: 99  prime 523  power: 46  ul[0]-power: 0  ulm: [357]  first: 138
n: 99  prime 523  power: 47  ul[0]-power: 0  ulm: [219]  first: 141
n: 99  prime 523  power: 48  ul[0]-power: 0  ulm: [288]  first: 144
n: 99  prime 523  power: 49  ul[0]-power: 0  ulm: [155]  first: 147
n: 99  prime 523  power: 50  ul[0]-power: 0  ulm: [40

n: 100  prime 541  power: 50  ul[0]-power: 0  ulm: [142]  first: 150
n: 100  prime 541  power: 51  ul[0]-power: 0  ulm: [251]  first: 153
n: 100  prime 541  power: 52  ul[0]-power: 0  ulm: [94]  first: 156
n: 100  prime 541  power: 53  ul[0]-power: 0  ulm: [323]  first: 159
n: 100  prime 541  power: 54  ul[0]-power: 0  ulm: [456]  first: 162
n: 100  prime 541  power: 55  ul[0]-power: 0  ulm: [375]  first: 165
n: 100  prime 541  power: 56  ul[0]-power: 0  ulm: [323]  first: 168
n: 100  prime 541  power: 57  ul[0]-power: 0  ulm: [479]  first: 171
n: 100  prime 541  power: 58  ul[0]-power: 0  ulm: [466]  first: 174
n: 100  prime 541  power: 59  ul[0]-power: 0  ulm: [14]  first: 177
n: 100  prime 541  power: 60  ul[0]-power: 0  ulm: [86]  first: 180
n: 100  prime 541  power: 61  ul[0]-power: 0  ulm: [289]  first: 183
n: 100  prime 541  power: 62  ul[0]-power: 0  ulm: [146]  first: 186
n: 100  prime 541  power: 63  ul[0]-power: 0  ulm: [110]  first: 189
n: 100  prime 541  power: 64  ul[0]-p